In [12]:
# synaptic bank

from carabao.util import repr
from carabao.neuron import Synapses,toy

k,g,K,P,c = toy('cell')
syn = Synapses(K)
c = [1,1,1,1,1,1,1,1,1,1]; print("c:",repr(c)) 

print("K   ",repr(K))
print("P   ",repr(P))
W = syn.W(P);   print("syn.W(P)  ",repr(W))
V = syn.V(c);   print("syn.V(c)  ",repr(V)) 
E = syn.E(c,P); print("syn.E(c,P)",repr(E))
S = syn.S(c,P); print("syn.S(c,P)",repr(S))
L = syn.L(c,S); print("syn.L(c,S)",repr(L))
s = syn.s(c,P); print("syn.s(c,P)",repr(s))
Q1 = syn.sat(1.5*P);  print("Q1",repr(Q1))
Q2 = syn.sat(P-0.7*Q1);  print("Q2",repr(Q2))

repr M: [[1 1 1 1 1 1 1 1 1 1]] <class 'numpy.ndarray'> (1, 10)
c: [1 1 1 1 1 1 1 1 1 1]
K    #[1 3 5 7 9; 5 6 7 8 9]
P    #[0.12 0.32 0.54 0.77 0; 0 0.61 0.45 0 0.8]
syn.W(P)   #[0 0 1 1 0; 0 1 0 0 1]
syn.V(c)   #[1 1 1 1 1; 1 1 1 1 1]
syn.E(c,P) #[0 0 1 1 0; 0 1 0 0 1]
syn.S(c,P) #[1 1 1 1 1; 1 1 1 1 1]
syn.L(c,S) #[0.02 0.02 0.02 0.02 0.02; 0.02 0.02 0.02 0.02 0.02]
syn.s(c,P) #[1 1]
Q1 #[0.18 0.48 0.81 1 0; 0 0.915 0.675 0 1]
Q2 #[0 0 0 0.07 0; 0 0 0 0 0.1]


In [2]:
# synaptic field

from carabao.cell import toy
k,g,K,P,c = toy('cell')
syn = Synapses(g)
c = [0,1,0,1,0,0,0,0,0,0]; print("c:",repr(c)) 
v=syn.v(c); print("syn.v(c)",repr(v)) 


c: [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
syn.v(c) array([0, 1, 0, 1])


In [4]:
help(Synapses)

Help on class Synapses in module carabao.neuron:

class Synapses(builtins.object)
 |  Synapses(K)
 |  
 |  Synapses: class
 |      syn = Synapses(g)              # construct synaptic field
 |      syn = Synapses(K)              # construct synaptic bank
 |  
 |      syn.parameter(eta,theta,(plus,minus))  # setup synaptic parameters
 |      syn.parameter()                # eta=0.5, theta=2, plus=minus=0.02
 |  
 |      K = syn.K                      # synaptic index matrix
 |      eta = syn.eta                  # synaptic threshold
 |      theta = syn.theta              # spiking threshold
 |      plus,minus = syn.delta         # learning deltas
 |  
 |      v = syn.v(c)                   # group activation
 |      s = syn.s(c,P)                 # spike vector
 |  
 |      V = syn.V(c)                   # presynaptic signals
 |      W = syn.W(P)                   # synaptic weight matrix
 |      E = syn.E(c,P)                 # empowering matrix
 |      S = syn.S(c,P)                 # 